In [1]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2

In [2]:
import gspread
from google.colab import auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials
import numpy as np
from scipy.stats import dirichlet
import statistics

# Authenticate the user
auth.authenticate_user()

import google.auth

# Get the authenticated credentials
creds, _ = google.auth.default()

# Authorize the gspread client with the credentials
client = gspread.authorize(creds)


In [3]:
# Replace with your Google Sheets URL
sheet_url = "https://docs.google.com/spreadsheets/d/125RYcsOnk7Eh-DMl-iJQqPhDwsGFXg-Ew3kUTd5w0iY/edit?gid=0#gid=0"

# Open the Google Sheet
sheet = client.open_by_url(sheet_url).sheet1  # Access the first worksheet

In [7]:
def read_parameters_scenario4():
    """
    Reads observed rolls for SUMA and SUMB from the Google Sheet.
    """
    try:
        # Read initial parameters
        total_rolls = 13
        rounds_revealed = int(sheet.acell('A10').value)
        teams = int(sheet.acell('C3').value)

        # Read observed rolls for SUMA and SUMB
        observed_rolls_A = []
        observed_rolls_B = []

        for minute in range(rounds_revealed):
            # Read observed roll for die A
            roll_A_cell = f'D{7 + minute}'  # Adjust if necessary
            roll_A_value = sheet.acell(roll_A_cell).value
            if roll_A_value:
                roll_A = int(roll_A_value)
                observed_rolls_A.append(roll_A)

            # Read observed roll for die B
            roll_B_cell = f'D{23 + minute}'  # Adjust if necessary
            roll_B_value = sheet.acell(roll_B_cell).value
            if roll_B_value:
                roll_B = int(roll_B_value)
                observed_rolls_B.append(roll_B)

        return observed_rolls_A, observed_rolls_B, total_rolls, teams
    except Exception as e:
        print(f"Error reading parameters: {e}")
        return [], [], 0, 0

def hierarchical_bayesian_update(observed_rolls, prior_alpha=None, beta=1.0):
    """
    Performs a hierarchical Bayesian update for die probabilities.
    Args:
        observed_rolls (list): List of observed rolls.
        prior_alpha (np.array): Prior alpha parameters for Dirichlet distribution.
        beta (float): Hyperparameter controlling the variability over time.
    Returns:
        posterior_probs (np.array): Posterior probabilities for die faces.
        posterior_alpha (np.array): Posterior Dirichlet parameters.
    """
    if prior_alpha is None:
        prior_alpha = np.ones(6)  # Uniform prior

    # Update the Dirichlet parameters with observed counts
    observed_counts = np.bincount(observed_rolls, minlength=7)[1:]  # Ignore zero index

    # Apply discounting to the prior to allow for time variability
    prior_alpha = beta * prior_alpha

    # Update posterior alpha
    posterior_alpha = prior_alpha + observed_counts

    # Calculate posterior probabilities
    posterior_probs = posterior_alpha / posterior_alpha.sum()

    return posterior_probs, posterior_alpha

def calculate_expected_sum_hierarchical(observed_rolls, posterior_probs, total_rolls):
    """
    Calculates the expected sum based on observed rolls and posterior probabilities.
    Args:
        observed_rolls (list): List of observed rolls.
        posterior_probs (np.array): Posterior probabilities for die faces.
        total_rolls (int): Total number of rolls in the game.
    Returns:
        expected_sum (float): Expected total sum.
    """
    # Current sum from observed rolls
    current_sum = sum(observed_rolls)

    # Remaining rolls
    remaining_rolls = total_rolls - len(observed_rolls)

    # Expected value per remaining roll
    expected_value_per_roll = np.dot(np.arange(1, 7), posterior_probs)

    # Expected sum from remaining rolls
    expected_remaining_sum = remaining_rolls * expected_value_per_roll

    # Total expected sum
    expected_sum = current_sum + expected_remaining_sum

    return expected_sum

def simulate_confidence_interval_hierarchical(observed_rolls, posterior_alpha, total_rolls, confidence_level=95, num_simulations=10000):
    """
    Simulates confidence intervals based on hierarchical Bayesian posterior.
    """
    remaining_rolls = total_rolls - len(observed_rolls)
    simulated_totals = []

    for _ in range(num_simulations):
        # Sample die probabilities from the posterior Dirichlet distribution
        sampled_probs = dirichlet.rvs(posterior_alpha, size=1)[0]

        # Simulate remaining rolls
        simulated_remaining_rolls = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs
        )

        # Calculate total sum
        simulated_total = sum(observed_rolls) + sum(simulated_remaining_rolls)
        simulated_totals.append(simulated_total)

    # Calculate confidence interval
    lower_percentile = (100 - confidence_level) / 2
    upper_percentile = 100 - lower_percentile
    lower_bound = np.percentile(simulated_totals, lower_percentile)
    upper_bound = np.percentile(simulated_totals, upper_percentile)

    return lower_bound, upper_bound

def simulate_spread_hierarchical(observed_rolls_A, posterior_alphaA, observed_rolls_B, posterior_alphaB, total_rolls, confidence_levels=[95, 80, 60], num_simulations=10000):
    """
    Simulates the spread (SPRD) as the absolute difference between SUMA and SUMB.
    """
    remaining_rolls = total_rolls - len(observed_rolls_A)  # Assuming same number of rolls for A and B

    simulated_spreads = []

    for _ in range(num_simulations):
        # Sample die probabilities for SUMA and SUMB
        sampled_probs_A = dirichlet.rvs(posterior_alphaA, size=1)[0]
        sampled_probs_B = dirichlet.rvs(posterior_alphaB, size=1)[0]

        # Simulate remaining rolls for SUMA and SUMB
        simulated_remaining_rolls_A = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs_A
        )
        simulated_remaining_rolls_B = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs_B
        )

        # Calculate total sums
        total_sum_A = sum(observed_rolls_A) + sum(simulated_remaining_rolls_A)
        total_sum_B = sum(observed_rolls_B) + sum(simulated_remaining_rolls_B)

        # Compute spread as absolute difference
        spread = abs(total_sum_A - total_sum_B)
        simulated_spreads.append(spread)

    simulated_spreads = np.array(simulated_spreads)

    # Compute statistics
    expected_spread = np.mean(simulated_spreads)
    variance_spread = np.var(simulated_spreads)
    std_spread = np.std(simulated_spreads)

    # Compute confidence intervals
    confidence_intervals = {}
    for cl in confidence_levels:
        lower_percentile = (100 - cl) / 2
        upper_percentile = 100 - lower_percentile
        lower_bound = max(np.percentile(simulated_spreads, lower_percentile), 0)
        upper_bound = np.percentile(simulated_spreads, upper_percentile)
        confidence_intervals[cl] = (lower_bound, upper_bound)

    return expected_spread, variance_spread, std_spread, confidence_intervals

def main_scenario4_hierarchical():
    # Read observed rolls
    observed_rolls_A, observed_rolls_B, total_rolls, teams = read_parameters_scenario4()

    if not observed_rolls_A or not observed_rolls_B:
        print("No observed rolls found. Exiting.")
        return

    # Initialize prior alpha
    prior_alpha_A = np.ones(6)
    prior_alpha_B = np.ones(6)

    # Hyperparameter beta controls how quickly we forget past observations
    beta = 0.9  # Adjust beta between 0 (forget quickly) and 1 (forget slowly)

    # Perform hierarchical Bayesian updates
    posterior_probsA, posterior_alphaA = hierarchical_bayesian_update(observed_rolls_A, prior_alpha_A, beta=beta)
    posterior_probsB, posterior_alphaB = hierarchical_bayesian_update(observed_rolls_B, prior_alpha_B, beta=beta)

    # Calculate expected sums
    expected_sum_A = calculate_expected_sum_hierarchical(observed_rolls_A, posterior_probsA, total_rolls)
    expected_sum_B = calculate_expected_sum_hierarchical(observed_rolls_B, posterior_probsB, total_rolls)

    # Simulate confidence intervals for SUMA
    lower_bound95A, upper_bound95A = simulate_confidence_interval_hierarchical(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=95, num_simulations=10000
    )
    lower_bound80A, upper_bound80A = simulate_confidence_interval_hierarchical(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=80, num_simulations=10000
    )
    lower_bound60A, upper_bound60A = simulate_confidence_interval_hierarchical(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=60, num_simulations=10000
    )

    # Simulate confidence intervals for SUMB
    lower_bound95B, upper_bound95B = simulate_confidence_interval_hierarchical(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=95, num_simulations=10000
    )
    lower_bound80B, upper_bound80B = simulate_confidence_interval_hierarchical(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=80, num_simulations=10000
    )
    lower_bound60B, upper_bound60B = simulate_confidence_interval_hierarchical(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=60, num_simulations=10000
    )

    # Simulate spread
    expected_spread, variance_spread, std_spread, confidence_intervals_spread = simulate_spread_hierarchical(
        observed_rolls_A, posterior_alphaA, observed_rolls_B, posterior_alphaB, total_rolls, confidence_levels=[95, 80, 60], num_simulations=10000
    )

    # Compute bid and ask prices
    z = 1.96  # Z-score for 95% confidence
    bid_price = max(expected_spread - z * std_spread, 0)
    ask_price = expected_spread + z * std_spread

    # Update Google Sheets with the results
    try:
      sheet.update([[expected_sum_A]], 'F8')
      # Update SUMA confidence intervals
      sheet.update([[lower_bound95A]], 'F13')
      sheet.update([[upper_bound95A]], 'H13')

      sheet.update([[lower_bound80A]], 'F19')
      sheet.update([[upper_bound80A]], 'H19')

      sheet.update([[lower_bound60A]], 'F25')
      sheet.update([[upper_bound60A]], 'H25')

      # Update SUMB expected sum
      sheet.update([[expected_sum_B]], 'F30')

      # Update SUMB confidence intervals
      sheet.update([[lower_bound95B]], 'F35')
      sheet.update([[upper_bound95B]], 'H35')

      sheet.update([[lower_bound80B]], 'F41')
      sheet.update([[upper_bound80B]], 'H41')

      sheet.update([[lower_bound60B]], 'F47')
      sheet.update([[upper_bound60B]], 'H47')

      # Update Spread statistics
      sheet.update([[expected_spread]], 'J30')
      sheet.update([[variance_spread]], 'K30')
      sheet.update([[std_spread]], 'L30')

      # Update Spread confidence intervals
      sheet.update([[confidence_intervals_spread[95][0]]], 'J35')
      sheet.update([[confidence_intervals_spread[95][1]]], 'L35')

      sheet.update([[confidence_intervals_spread[80][0]]], 'J41')
      sheet.update([[confidence_intervals_spread[80][1]]], 'L41')

      sheet.update([[confidence_intervals_spread[60][0]]], 'J47')
      sheet.update([[confidence_intervals_spread[60][1]]], 'L47')

      # Update Bid and Ask Prices
      sheet.update([[bid_price]], 'J50')
      sheet.update([[ask_price]], 'L50')

      print("Successfully updated Google Sheets with the latest calculations.")
    except Exception as e:
        print(f"An error occurred while updating Google Sheets: {e}")

# Execute the main function for Scenario 4
main_scenario4_hierarchical()


Successfully updated Google Sheets with the latest calculations.
